# Topics


In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from z3 import *
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("14-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'p=46,91 v=80,-6\np=89,91 v=-69,68\np=61,39 v=-75,66\np=8,63 v=48,-75\np=58,76 v=-18,19\np=99,14 v=-92,-85\np=80,66 v=-3,-13\np=15,1 v=-11,30\np=52,79 v=93,81\np=81,44 v=24,99\np=6,41 v=-12,61\np=40,14 v=-47,-85\np=53,30 v=-52,-78\np=2,53 v=-97,-33\np=25,75 v=79,44\np=77,17 v=-30,-21\np=41,41 v=93,-99\np=95,98 v=-49,43\np=52,56 v=-88,-55\np=99,20 v=-64,-53\np=9,7 v=-56,-26\np=18,0 v=87,-77\np=90,95 v=-51,72\np=25,81 v=20,-39\np=94,79 v=-12,68\np=58,88 v=-88,6\np=31,99 v=-11,-85\np=99,79 v=33,-46\np=86,25 v=97,-17\np=79,73 v=-35,-4\np=20,52 v=17,63\np=65,65 v=-98,-34\np=34,31 v=33,-49\np=14,21 v=17,54\np=71,16 v=29,42\np=63,53 v=-41,44\np=59,78 v=-44,-92\np=44,96 v=88,93\np=88,70 v=-38,36\np=78,78 v=16,11\np=12,35 v=27,21\np=45,77 v=-65,-10\np=80,87 v=31,89\np=31,2 v=-68,59\np=24,64 v=-6,3\np=28,34 v=54,-98\np=99,61 v=-71,-21\np=11,84 v=-37,60\np=14,14 v=-68,-57\np=97,12 v=-92,58\np=25,69 v=39,-61\np=72,34 v=60,-62\np=15,84 v=-10,14\np=96,57 v=-56,15\np=76,55 v=-33,98\np=23,97 v=56,60\n

In [7]:
test_data_raw = r"""p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3"""



Robot = namedtuple("Robot", ["px", "py","vx", "vy"])

def preprocess_data (data):
    
    def prep_setup(a):
        
        p0, p1, v0, v1 = re.findall(r'-?\d+', a)
        
        return Robot(px=int(p0), py=int(p1), vx=int(v0), vy=int(v1))
    
    rows = [prep_setup(row) for row in data.split("\n")]
    return rows



test_data = preprocess_data(test_data_raw)

display(test_data)


[Robot(px=0, py=4, vx=3, vy=-3),
 Robot(px=6, py=3, vx=-1, vy=-3),
 Robot(px=10, py=3, vx=-1, vy=2),
 Robot(px=2, py=0, vx=2, vy=-1),
 Robot(px=0, py=0, vx=1, vy=3),
 Robot(px=3, py=0, vx=-2, vy=-2),
 Robot(px=7, py=6, vx=-1, vy=-3),
 Robot(px=3, py=0, vx=-1, vy=-2),
 Robot(px=9, py=3, vx=2, vy=3),
 Robot(px=7, py=3, vx=-1, vy=2),
 Robot(px=2, py=4, vx=2, vy=-3),
 Robot(px=9, py=5, vx=-3, vy=-3)]

In [8]:
full_data = preprocess_data(data_raw)
full_data

[Robot(px=46, py=91, vx=80, vy=-6),
 Robot(px=89, py=91, vx=-69, vy=68),
 Robot(px=61, py=39, vx=-75, vy=66),
 Robot(px=8, py=63, vx=48, vy=-75),
 Robot(px=58, py=76, vx=-18, vy=19),
 Robot(px=99, py=14, vx=-92, vy=-85),
 Robot(px=80, py=66, vx=-3, vy=-13),
 Robot(px=15, py=1, vx=-11, vy=30),
 Robot(px=52, py=79, vx=93, vy=81),
 Robot(px=81, py=44, vx=24, vy=99),
 Robot(px=6, py=41, vx=-12, vy=61),
 Robot(px=40, py=14, vx=-47, vy=-85),
 Robot(px=53, py=30, vx=-52, vy=-78),
 Robot(px=2, py=53, vx=-97, vy=-33),
 Robot(px=25, py=75, vx=79, vy=44),
 Robot(px=77, py=17, vx=-30, vy=-21),
 Robot(px=41, py=41, vx=93, vy=-99),
 Robot(px=95, py=98, vx=-49, vy=43),
 Robot(px=52, py=56, vx=-88, vy=-55),
 Robot(px=99, py=20, vx=-64, vy=-53),
 Robot(px=9, py=7, vx=-56, vy=-26),
 Robot(px=18, py=0, vx=87, vy=-77),
 Robot(px=90, py=95, vx=-51, vy=72),
 Robot(px=25, py=81, vx=20, vy=-39),
 Robot(px=94, py=79, vx=-12, vy=68),
 Robot(px=58, py=88, vx=-88, vy=6),
 Robot(px=31, py=99, vx=-11, vy=-85),
 Rob

In [57]:

def solution (data, verbose=False):
    robots = data[0]
    tiles_shape = data[1]
    
    
    def get_quadrant(size, x, y):
        if x < size[0] // 2 and y < size[1] // 2:
            return "NW"
        elif x < size[0] // 2 and y > size[1] // 2:
            return "NE"
        elif x > size[0] // 2 and y > size[1] // 2:
            return "SE"
        elif x > size[0] // 2 and y < size[1] // 2:
            return "SW"
        else:
            return "middle"

    def field_after_seconds (seconds):
        field = np.full(tiles_shape, ".")

        quadrants = {"NW": 0, "NE":0, "SE":0, "SW":0}
        middle_robots=0
        for robot in robots:

            # px, vx: columns, py,vy: rows
            y = (robot.px + robot.vx* seconds  ) % tiles_shape[1]
            x = (robot.py + robot.vy* seconds  ) % tiles_shape[0]
            quad = get_quadrant(tiles_shape, x, y)
            if quad != 'middle':
                quadrants[quad] += 1
            else:
                middle_robots+=1
            
            if field[x,y] == '.':
                field[x,y] = '1'
            else: 
                field[x,y] = str(int(field[x,y]) + 1)
                
        return field, quadrants
    
    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str
   
    
    seconds = 100
    field, quadrants = field_after_seconds(100)
    
    start_field, start_quadrants = field_after_seconds(0)
    # print(display_field(start_field))
    # print("\n")
    # print(display_field(field))
    print(quadrants)
    
    return np.prod(list(quadrants.values()))


# 7 rows, 11 columns
sol = solution((test_data, (7,11)), verbose=True)
print(sol)





{'NW': 1, 'NE': 3, 'SE': 1, 'SW': 4}
12


In [58]:
sol = solution(([Robot(px=2, py=4, vx=2, vy=-3)], (7,11)), verbose=True)


{'NW': 0, 'NE': 0, 'SE': 0, 'SW': 1}


In [59]:
sol = solution((full_data, (103,101)))
print(sol)

{'NW': 121, 'NE': 119, 'SE': 121, 'SW': 132}
229980828


# Part 2

In [83]:
# Doesn't work because of rounding errors 
# Used z3 solution

def solution2 (data, verbose=False):
    robots = data[0]
    tiles_shape = data[1]
    
    
    def get_quadrant(size, x, y):
        if x < size[0] // 2 and y < size[1] // 2:
            return "NW"
        elif x < size[0] // 2 and y > size[1] // 2:
            return "NE"
        elif x > size[0] // 2 and y > size[1] // 2:
            return "SE"
        elif x > size[0] // 2 and y < size[1] // 2:
            return "SW"
        else:
            return "middle"

    def field_after_seconds (seconds):
        field = np.full(tiles_shape, ".")

        quadrants = {"NW": 0, "NE":0, "SE":0, "SW":0}
        middle_robots=0
        for robot in robots:

            # px, vx: columns, py,vy: rows
            y = (robot.px + robot.vx* seconds  ) % tiles_shape[1]
            x = (robot.py + robot.vy* seconds  ) % tiles_shape[0]
            quad = get_quadrant(tiles_shape, x, y)
            if quad != 'middle':
                quadrants[quad] += 1
            else:
                middle_robots+=1
            
            if field[x,y] == '.':
                field[x,y] = '1'
            else: 
                field[x,y] = str(int(field[x,y]) + 1)
                
        return field, quadrants
    
    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str
   
    
    middle_row_count = []
    open('robots.txt', 'w').close()
    for s in tqdm(range(100000)):
        f, q = field_after_seconds(s)
        middle_x =  tiles_shape[0] // 2
        middle_y =  tiles_shape[1] // 2
        count = len([r  for r in f[:, middle_y].tolist() if r != '.'])

        middle_row_count.append(count)
        
        with open("robots.txt", "a") as myfile:
            myfile.write(f"{str(s)}\n")
            myfile.write(display_field(f))
            myfile.write("\n")
         
    seconds = 100
    field, quadrants = field_after_seconds(100)
    
    
    return np.prod(list(quadrants.values()))


# sol = solution2((test_data, (7,11)), verbose=True)
# print("Solution", sol)
    

In [84]:
sol = solution2((full_data, (103,101)))
print("Solution", sol)

  0%|          | 0/100000 [00:00<?, ?it/s]

100%|██████████| 100000/100000 [01:50<00:00, 908.79it/s]

[3, 3, 4, 8, 3, 3, 8, 1, 6, 3, 6, 6, 4, 6, 7, 6, 4, 6, 4, 6, 6, 5, 6, 9, 5, 6, 5, 5, 1, 4, 3, 6, 5, 5, 4, 4, 5, 6, 5, 4, 3, 5, 2, 7, 4, 5, 3, 7, 2, 7, 6, 3, 5, 4, 3, 6, 5, 4, 4, 8, 3, 2, 13, 6, 9, 6, 3, 2, 2, 8, 4, 7, 3, 5, 4, 9, 5, 9, 2, 6, 8, 1, 7, 2, 2, 5, 8, 2, 4, 6, 5, 8, 5, 4, 2, 2, 6, 2, 4, 7, 4, 4, 3, 4, 8, 3, 3, 8, 1, 7, 3, 6, 6, 3, 6, 7, 6, 4, 6, 4, 6, 6, 5, 6, 9, 5, 6, 4, 5, 1, 4, 3, 6, 5, 5, 4, 4, 5, 6, 5, 4, 3, 5, 2, 8, 4, 5, 3, 7, 3, 7, 6, 3, 6, 4, 3, 6, 5, 4, 4, 7, 3, 2, 13, 6, 9, 6, 3, 3, 2, 8, 4, 7, 3, 5, 4, 9, 5, 9, 2, 5, 8, 1, 7, 2, 2, 5, 7, 2, 4, 5, 5, 9, 5, 4, 2, 2, 6, 2, 4, 7, 4, 4, 3, 4, 8, 3, 3, 8, 1, 6, 3, 6, 6, 4, 6, 7, 5, 4, 6, 4, 6, 6, 4, 6, 9, 5, 6, 5, 5, 1, 4, 3, 6, 5, 5, 4, 4, 5, 6, 5, 4, 3, 5, 2, 8, 4, 5, 3, 7, 3, 7, 6, 3, 6, 4, 3, 6, 5, 4, 4, 8, 3, 2, 11, 6, 8, 6, 3, 3, 2, 8, 4, 7, 3, 5, 4, 8, 5, 9, 2, 6, 8, 1, 7, 2, 2, 5, 8, 2, 5, 6, 4, 8, 5, 4, 2, 2, 6, 2, 3, 7, 4, 4, 3, 4, 8, 3, 3, 6, 1, 7, 3, 6, 6, 4, 6, 7, 6, 4, 6, 4, 6, 6, 5, 6, 9, 5, 6, 5, 5, 1, 